In [3]:
import flwr as fl
from typing import Dict, Optional, Tuple, List
from flwr.common import Metrics, Parameters, Scalar, FitRes, EvaluateRes
import logging
from datetime import datetime
import time

In [4]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("Server")

def log_status(status: str, details: str = ""):
    status_line = f"\n{'='*20} {status} {'='*20}"
    logger.info(status_line)
    if details:
        logger.info(details)
    logger.info("="*len(status_line))
    print(status_line)
    if details:
        print(details)
    print("="*len(status_line))

class LoggingStrategy(fl.server.strategy.FedAvg):
    def __init__(
        self,
        *,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 1,
        min_evaluate_clients: int = 1,
        min_available_clients: int = 1,
        timeout: Optional[float] = None,
    ) -> None:
        super().__init__(
            fraction_fit=fraction_fit,
            fraction_evaluate=fraction_evaluate,
            min_fit_clients=min_fit_clients,
            min_evaluate_clients=min_evaluate_clients,
            min_available_clients=min_available_clients,
        )
        self.round_metrics = []
        self.expected_clients = min_available_clients
        self.timeout = timeout
        self.current_round = 0
        self.connected_clients = set()

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitIns]]:
        self.current_round = server_round
        connected = client_manager.num_available()
        
        # Get newly connected clients
        current_clients = set(client_manager.all().keys())
        new_clients = current_clients - self.connected_clients
        self.connected_clients.update(new_clients)
        
        if new_clients:
            log_status("NEW CLIENT CONNECTED", 
                      f"Round {server_round}\n"
                      f"Total connected: {connected}/{self.expected_clients}\n"
                      f"New client(s): {len(new_clients)}\n"
                      f"Waiting for {self.expected_clients - connected} more")
        
        if connected < self.expected_clients:
            log_status("WAITING", 
                      f"Round {server_round}: Waiting for more clients\n"
                      f"Currently connected: {connected}/{self.expected_clients}\n"
                      f"You can start {self.expected_clients - connected} more client(s)")
            # Sleep to prevent too frequent logging
            time.sleep(2)
        else:
            log_status("STARTING ROUND", 
                      f"Round {server_round}: All {self.expected_clients} clients connected\n"
                      f"Beginning training round")
        
        return super().configure_fit(server_round, parameters, client_manager)

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        log_status("ROUND COMPLETE", 
                  f"Round {server_round} completed\n"
                  f"Successful clients: {len(results)}\n"
                  f"Failed clients: {len(failures)}")
        
        # Store metrics
        round_data = {
            "round": server_round,
            "timestamp": datetime.now().isoformat(),
            "num_clients": len(results),
            "num_failures": len(failures)
        }
        self.round_metrics.append(round_data)
        
        return super().aggregate_fit(server_round, results, failures)

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.EvaluateIns]]:
        log_status("EVALUATION", f"Round {server_round}: Starting evaluation")
        return super().configure_evaluate(server_round, parameters, client_manager)

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        log_status("EVALUATION COMPLETE", 
                  f"Round {server_round}\n"
                  f"Successful evaluations: {len(results)}\n"
                  f"Failed evaluations: {len(failures)}")
        return super().aggregate_evaluate(server_round, results, failures)

# Define a larger message size (1GB)
GRPC_MAX_MESSAGE_LENGTH = 1024 * 1024 * 1024

# Initialize strategy with longer timeout
strategy = LoggingStrategy(
    min_fit_clients=1,
    min_available_clients=1,
    min_evaluate_clients=1,
    timeout=None  # No timeout, wait indefinitely for clients
)

log_status("SERVER STARTING", 
          "Initializing Flower server\n"
          f"Address: 127.0.0.1:8081\n"
          f"Expected clients: 1\n"
          f"Message size: {GRPC_MAX_MESSAGE_LENGTH}\n"
          f"You can now start connecting clients one by one")

# Start server
try:
    fl.server.start_server(
        server_address="127.0.0.1:8081",
        config=fl.server.ServerConfig(
            num_rounds=3,
            round_timeout=None  # No timeout for rounds
        ),
        grpc_max_message_length=GRPC_MAX_MESSAGE_LENGTH,
        strategy=strategy
    )
    
    log_status("SERVER COMPLETED", 
              f"Training completed successfully\n"
              f"Total rounds: {len(strategy.round_metrics)}")
    
    # Print final metrics
    print("\nTraining Round Metrics:")
    for round_data in strategy.round_metrics:
        print(f"\nRound {round_data['round']}:")
        print(f"Timestamp: {round_data['timestamp']}")
        print(f"Active clients: {round_data['num_clients']}")
        print(f"Failed clients: {round_data['num_failures']}")
    
except Exception as e:
    log_status("SERVER ERROR", str(e))
finally:
    log_status("SERVER SHUTDOWN", "Flower server has shut down")

2025-03-09 15:13:03,252 - Server - INFO - 
==================== SERVER STARTING ====================
2025-03-09 15:13:03,253 - Server - INFO - Initializing Flower server
Address: 127.0.0.1:8081
Expected clients: 1
Message size: 1073741824
You can now start connecting clients one by one
2025-03-09 15:13:03,254 - Server - INFO - ==========================================================
	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
2025-03-09 15:13:03,255 - flwr - WARNING - DEPRECATED FEATURE: flwr.server.start_server() is deprecated.
	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all availa


==================== SERVER STARTING ====================
Initializing Flower server
Address: 127.0.0.1:8081
Expected clients: 1
Message size: 1073741824
You can now start connecting clients one by one


INFO :      Received initial parameters from one random client
2025-03-09 15:13:48,077 - flwr - INFO - Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
2025-03-09 15:13:48,080 - flwr - INFO - Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
2025-03-09 15:13:48,082 - flwr - INFO - Evaluation returned no results (`None`)
INFO :      
2025-03-09 15:13:48,083 - flwr - INFO - 
INFO :      [ROUND 1]
2025-03-09 15:13:48,084 - flwr - INFO - [ROUND 1]
2025-03-09 15:13:48,085 - Server - INFO - 
==================== NEW CLIENT CONNECTED ====================
2025-03-09 15:13:48,086 - Server - INFO - Round 1
Total connected: 1/1
New client(s): 1
Waiting for 0 more
2025-03-09 15:13:48,086 - Server - INFO - ===============================================================
2025-03-09 15:13:48,087 - Server - INFO - 
==================== STARTING ROUND ====================
2025-03-09 15:13:48


==================== NEW CLIENT CONNECTED ====================
Round 1
Total connected: 1/1
New client(s): 1
Waiting for 0 more

==================== STARTING ROUND ====================
Round 1: All 1 clients connected
Beginning training round


INFO :      aggregate_fit: received 1 results and 0 failures
2025-03-09 15:21:18,601 - flwr - INFO - aggregate_fit: received 1 results and 0 failures
2025-03-09 15:21:18,611 - Server - INFO - 
==================== ROUND COMPLETE ====================
2025-03-09 15:21:18,612 - Server - INFO - Round 1 completed
Successful clients: 1
Failed clients: 0
2025-03-09 15:21:18,613 - Server - INFO - =========================================================


...


==================== ROUND COMPLETE ====================
Round 3 completed
Successful clients: 1
Failed clients: 0


2025-03-09 15:30:26,634 - Server - INFO - 
==================== EVALUATION ====================
2025-03-09 15:30:26,635 - Server - INFO - Round 3: Starting evaluation
2025-03-09 15:30:26,636 - Server - INFO - =====================================================
INFO :      configure_evaluate: strategy sampled 1 clients (out of 1)
2025-03-09 15:30:26,637 - flwr - INFO - configure_evaluate: strategy sampled 1 clients (out of 1)



==================== EVALUATION ====================
Round 3: Starting evaluation


INFO :      aggregate_evaluate: received 0 results and 1 failures
2025-03-09 15:30:38,578 - flwr - INFO - aggregate_evaluate: received 0 results and 1 failures
2025-03-09 15:30:38,580 - Server - INFO - 
==================== EVALUATION COMPLETE ====================
2025-03-09 15:30:38,580 - Server - INFO - Round 3
Successful evaluations: 0
Failed evaluations: 1
2025-03-09 15:30:38,581 - Server - INFO - ==============================================================
INFO :      
2025-03-09 15:30:38,692 - flwr - INFO - 
INFO :      [SUMMARY]
2025-03-09 15:30:38,695 - flwr - INFO - [SUMMARY]
INFO :      Run finished 3 round(s) in 1010.54s
2025-03-09 15:30:38,696 - flwr - INFO - Run finished 3 round(s) in 1010.54s
INFO :      
2025-03-09 15:30:38,698 - flwr - INFO - 
2025-03-09 15:30:38,738 - Server - INFO - 
==================== SERVER COMPLETED ====================
2025-03-09 15:30:38,744 - Server - INFO - Training completed successfully
Total rounds: 3
2025-03-09 15:30:38,744 - Server - I


==================== EVALUATION COMPLETE ====================
Round 3
Successful evaluations: 0
Failed evaluations: 1

==================== SERVER COMPLETED ====================
Training completed successfully
Total rounds: 3

Training Round Metrics:

Round 1:
Timestamp: 2025-03-09T15:21:18.614139
Active clients: 1
Failed clients: 0

Round 2:
Timestamp: 2025-03-09T15:25:50.403659
Active clients: 1
Failed clients: 0

Round 3:
Timestamp: 2025-03-09T15:30:25.821534
Active clients: 1
Failed clients: 0

==================== SERVER SHUTDOWN ====================
Flower server has shut down
